In [31]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from matplotlib import colors
import pygmt
#from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import time

In [32]:
import sys
sys.path.append(r'../tools_F-J/')
#from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard.mathlib import GetStationPairs
#from toollib_standard import filelib
#from toollib_standard import stacklib
from toollib_standard import plotlib

In [33]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)

dir_CC_workspace:  ./
dir_project_workspace:  ./


In [34]:
name_projects = []
name_projects.append('project/output_regular_v9.5_5x13/')
name_projects.append('project_repartition_v4.0/output_repar_v9.5_02--10-16Hz/')

In [35]:
name_project_sample = 'project_repartition_v4.0/output_repar_v9.5_02--10-16Hz/'
dir_project_sample = dir_project_workspace + name_project_sample
filename = dir_project_sample+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
key_subworks_all = info_basic['key_subworks']
txtfile = dir_project_sample+'improve.txt'
nums = np.loadtxt(txtfile,dtype='int')  
nums

array([ 86, 108, 121, 142, 152, 167, 176, 178, 220, 222, 224, 235, 246,
       276, 307, 317, 329, 373, 404, 437, 442, 520, 521, 535, 549, 568,
       587, 592])

In [36]:
key_subworks = []
for num in nums:
    tag = str(num)
    for key_subwork in key_subworks_all:
        if tag == key_subwork.split('--')[0]:
            key_subworks.append(key_subwork)
key_subworks

['86--37-05',
 '121--37-06',
 '167--43-07',
 '176--52-07',
 '178--07-08',
 '220--08-09',
 '222--10-09',
 '224--13-09',
 '235--26-09',
 '307--08-11',
 '317--24-11',
 '329--37-11',
 '373--37-12',
 '437--08-14',
 '442--18-14',
 '520--33-16',
 '521--34-16',
 '535--48-16',
 '568--53-17',
 '587--39-18']

In [37]:
dir_image = name_project_sample + 'compare_relative_error_Re/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)

In [38]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_stations_all = stainfo['latitude'].tolist() 
lon_stations_all = stainfo['longitude'].tolist()

In [39]:
stations_alls = {}
lat_stations_alls = {}
lon_stations_alls = {}
lat_centroid_alls = {}
lon_centroid_alls = {}
for name_project in name_projects:
    dir_project = dir_project_workspace + name_project
    filename = dir_project+'Basic_info.yml'
    with open(filename, 'r', encoding='utf-8') as f:
        info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
    dir_partition = dir_project + 'partition/'
    stations = {}
    lat = {}
    lon = {}
    lat_centroid = {}
    lon_centroid = {}
    for key in info_basic['key_subworks']:
        filepath = dir_partition + str(key) + '.txt'
        stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
        stations[key] = stations_this.tolist()
        lat[key] = lat_stations_this.astype(float)
        lon[key] = lon_stations_this.astype(float)
        lat_centroid[key] = np.mean(lat[key])
        lon_centroid[key] = np.mean(lon[key])

    stations_alls[name_project] = stations
    lat_stations_alls[name_project] = lat
    lon_stations_alls[name_project] = lon
    lat_centroid_alls[name_project] = lat_centroid
    lon_centroid_alls[name_project] = lon_centroid  

In [40]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()
name_ref, lat_fault_infer, lon_fault_infer = np.loadtxt('clark_fault_inferred.txt',dtype='str' , unpack=True,delimiter=',')
lon_fault_infer = lon_fault_infer.astype(float)
lat_fault_infer = lat_fault_infer.astype(float)

In [41]:
def find_nearset(key_subwork,name_project):
	global name_project_sample

	lon_this = lon_centroid_alls[name_project_sample][key_subwork]
	lat_this = lat_centroid_alls[name_project_sample][key_subwork]

	lon_stations_all = []
	lat_stations_all = []
	
	lon_stations_all = list(lon_centroid_alls[name_project].values())
	lat_stations_all = list(lat_centroid_alls[name_project].values())
	key_all = list(lat_centroid_alls[name_project].keys())

	dist = np.sqrt((np.array(lon_stations_all)-lon_this)**2+(np.array(lat_stations_all)-lat_this)**2)
	indx = np.argsort(dist)
	#print(indx)
	key_nearset = key_all[indx[0]]

	return str(key_nearset)

In [97]:
def plot_compare(key_subwork):
    global name_projects
    c_map = 'jet'
    v_min = 0.1
    v_max = None
    xlim_f = [5,20]
    ylim_c = [0.2,2]
    print('start ',key_subwork)

    # plot PST
    name_project =name_projects[1]
    key_this_find = find_nearset(key_subwork,name_project)
    stations = stations_alls[name_project]
    lat_stations = lat_stations_alls[name_project]
    lon_stations = lon_stations_alls[name_project]
    dir_project = dir_project_workspace + name_project
    filename = dir_project+'Basic_info.yml'
    with open(filename, 'r', encoding='utf-8') as f:
        info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
    dir_ds = dir_project + info_basic['rdir_ds']
    stalist = stations[key_this_find]
    lat_PST = lat_stations[key_this_find]
    lon_PST = lon_stations[key_this_find]
    name_project = name_projects[0]
    c = np.linspace(info_basic['fj_c_min'],info_basic['fj_c_max'],info_basic['fj_c_num'])
    ds = h5py.File(dir_ds+'ds_'+str(key_this_find)+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    ds_linear = ds['ds_linear'][0]
    ds_linear = plotlib.smooth_ds(ds_linear)
    ds_remove = plotlib.smooth_ds(ds_remove,level_f=10,level_c=15)
    ds_remove = np.nan_to_num(ds_remove)
    f = ds['f'][:]
    #ds.close()
    fig = plt.figure(figsize=(9.5,7.5))
    ax = fig.add_subplot(1,1,1)
    title = ' '
    im,ax = plotlib.plot_fj(ax,ds_remove,title,f,c,0,v_min=v_min,v_max=v_max,c_map=c_map,flag_im= 1)
    ax.set_xlim(xlim_f)
    ax.set_xticks([])
    ax.set_ylim(ylim_c)
    ax.set_xticks(np.arange(6,20,2))
    ax.tick_params(axis='both', which='major', width=2, length=5)  # 主刻度线宽度设为2，长度为5
    #ax.set_xticklabels([])
    #刻度设置在右边
    ax.yaxis.set_label_position("right")  # 将y轴的标签放到右侧
    ax.yaxis.set_ticks_position('right')  # 将y轴的刻度放到右侧
    #ax.yaxis.tick_right()
    #plt.rcParams['ytick.labelright'] = True
    ax.set_ylabel('Vel (km/s)')
    ax.set_yticks([0.5,1,1.5,2])
    #ax.set_xlabel('Frequency (Hz)')
    fig.colorbar(im, extend = 'both',orientation = 'horizontal',pad=0.1,shrink = 0.8, aspect = 30)
    plt.tight_layout()
    plt.savefig(dir_image+key_subwork+'_PST.tiff')
    plt.close()

    # plot regular
    name_project =name_projects[0]
    key_this_find = find_nearset(key_subwork,name_project)
    stations = stations_alls[name_project]
    lat_stations = lat_stations_alls[name_project]
    lon_stations = lon_stations_alls[name_project]
    dir_project = dir_project_workspace + name_project
    filename = dir_project+'Basic_info.yml'
    with open(filename, 'r', encoding='utf-8') as f:
        info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
    dir_ds = dir_project + info_basic['rdir_ds']
    stalist = stations[key_this_find]
    lat_regular = lat_stations[key_this_find]
    lon_regular = lon_stations[key_this_find]
    name_project = name_projects[0]
    c = np.linspace(info_basic['fj_c_min'],info_basic['fj_c_max'],info_basic['fj_c_num'])
    ds = h5py.File(dir_ds+'ds_'+str(key_this_find)+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    ds_linear = ds['ds_linear'][0]
    ds_linear = plotlib.smooth_ds(ds_linear)
    ds_remove = plotlib.smooth_ds(ds_remove,level_f=10,level_c=5)
    ds_remove = np.nan_to_num(ds_remove)
    f = ds['f'][:]
    #ds.close()
    #fig, ax = plt.subplots()
    fig = plt.figure(figsize=(8.5,7.5))
    ax = fig.add_subplot(1,1,1)
    title = ' '
    im,ax = plotlib.plot_fj(ax,ds_remove,title,f,c,0,v_min=v_min,v_max=v_max,c_map=c_map,flag_im = 1)
    ax.set_xlim(xlim_f)
    ax.set_ylim(ylim_c)
    ax.set_yticks([0.5,1,1.5,2])
    ax.set_xticks(np.arange(6,20,2))
    ax.tick_params(axis='both', which='major', width=2, length=5)  # 主刻度线宽度设为2，长度为5
    #ax.set_xticklabels([])
    ax.set_yticklabels([])
    #刻度设置在右边
    ax.yaxis.set_label_position("right")  # 将y轴的标签放到右侧
    ax.yaxis.set_ticks_position('right')  # 将y轴的刻度放到右侧
    fig.colorbar(im, extend = 'both',orientation = 'horizontal',pad=0.1,shrink = 0.8, aspect = 30)
    #ax.set_xlabel('Frequency (Hz)')
    plt.tight_layout()
    plt.savefig(dir_image+key_subwork+'_regular.tiff')
    plt.close()

    # plot partitions
    flag_red = 0.001
    flag_proj = 'M10c'
    fig = pygmt.Figure()
    fig.basemap(
        region=[np.min(lon_stations_all)-flag_red, np.max(lon_stations_all)+flag_red, np.min(lat_stations_all)-flag_red, np.max(lat_stations_all)+flag_red],
        projection=flag_proj, 
        frame='f',
    )
    # stations and faults
    fig.coast(region=[np.min(lon_stations_all)-flag_red, np.max(lon_stations_all)+flag_red, np.min(lat_stations_all)-flag_red, np.max(lat_stations_all)+flag_red],projection=flag_proj, frame='f', land="gray")
    fig.plot(x=lon_stations_all, y=lat_stations_all, fill='black', style='c0.06')
    for j in range(len(faults)):
        if j == 0:
            fig.plot(x=faults['clark'+str(j+1)]['lon'], y=faults['clark'+str(j+1)]['lat'], pen = "1.5p,black")
        else:
            fig.plot(x=faults['clark'+str(j+1)]['lon'], y=faults['clark'+str(j+1)]['lat'],pen = "1.5p,black")
    fig.plot(x=lon_fault_infer, y=lat_fault_infer, pen = "1.5p,black,-")
    # original partition
    fig.plot(x=lon_PST, y=lat_PST, style='c0.12',pen='2p,blue')
    fig.plot(x=lon_regular, y=lat_regular, style='t0.14',fill = 'green')
    #fig.plot(x=lon_stations_partition[key_subwork], y=lat_stations_partition[key_subwork],style='c0.12',pen='2p,blue')
    #fig.plot(x=lon_centroid_targets[targets.index(key_target)], y=lat_centroid_targets[targets.index(key_target)], style='t0.2',fill = 'red')
    #fig.plot(x=np.mean(lon_stations_partition[key_subwork]),y = np.mean(lat_stations_partition[key_subwork]), style='a0.2',fill = 'red')
    #fig.show()
    fig.savefig(dir_image+key_subwork+'_partition.tiff')


In [99]:
plt.style.use('tomo_paper_compare-fj.mplstyle')
#plot_compare('535--48-16')
for key_subwork in key_subworks:
    plot_compare(key_subwork)

start  86--37-05
Input file size is 1205, 1093
0...10...20...30...40...50...60...70...80...90...100 - done.
start  121--37-06
Input file size is 1205, 1093
0...10...20...30...40...50...60...70...80...90...100 - done.
start  167--43-07
Input file size is 1205, 1093
0...10...20...30...40...50...60...70...80...90...100 - done.
start  176--52-07
Input file size is 1205, 1093
0...10...20...30...40...50...60...70...80...90...100 - done.
start  178--07-08
Input file size is 1205, 1093
0...10...20...30...40...50...60...70...80...90...100 - done.
start  220--08-09
Input file size is 1205, 1093
0...10...20...30...40...50...60...70...80...90...100 - done.
start  222--10-09
Input file size is 1205, 1093
0...10...20...30...40...50...60...70...80...90...100 - done.
start  224--13-09
Input file size is 1205, 1093
0...10...20...30...40...50...60...70...80...90...100 - done.
start  235--26-09
Input file size is 1205, 1093
0...10...20...30...40...50...60...70...80...90...100 - done.
start  307--08-11
In